In [1]:
import sys
import re
import os
import numpy as np
import pandas as pd
import pysam
import matplotlib.pyplot as plt
import seaborn as sns
import pysam
from Bio.Seq import Seq
import gget
import mappy as mp

In [2]:
# get alignments from bamfiles

fpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/merged_bam/b01.GRCm39.bam"
bamfile = pysam.AlignmentFile(fpath, "rb")


res = []
for align in bamfile:

    read_name = align.qname
    offset = 0
    is_digested = False
    flag = '-1'
        
    if "_" in read_name:
        read_name = read_name.split("_")[0]
        offset = int(align.qname.split("_")[2])
        flag = int(align.qname.split("_")[1])
        is_digested = True
        
    row = {
        'read_name' : read_name,
        'is_digested' : is_digested,
        'mapping_quality' : align.mapping_quality,
        'flag' : flag,
        'chrom' : align.reference_name,
        'read_start' : align.qstart + offset,
        'read_end' : align.qend + offset,
        'ref_start' : align.reference_start,
        'ref_end' : align.reference_end,
    }

    res.append(row)

res = pd.DataFrame(res)
res = res.drop_duplicates()
res['align_id'] = res.groupby('read_name').cumcount() + 1
print(f"{res.shape=}")
res.head()

[E::hts_open_format] Failed to open file "/scratch/indikar_root/indikar1/cstansbu/scpc_test/merged_bam/b01.GRCm39.bam" : No such file or directory


FileNotFoundError: [Errno 2] could not open alignment file `/scratch/indikar_root/indikar1/cstansbu/scpc_test/merged_bam/b01.GRCm39.bam`: No such file or directory

In [ ]:
break

In [ ]:
fpath = "/scratch/indikar_root/indikar1/cstansbu/scpc_test/sequence_reports/b01.report.pq"

df = pd.read_parquet(fpath)
print(f"{df.shape=}")
df.head()

In [ ]:
# how many map?
df['is_mapped'] = df['read_name'].isin(res['read_name'].unique())
df['is_mapped'].value_counts(normalize=True)

In [ ]:
# how many map perfectly?

perf = res[res['mapping_quality'] > 0]

df['is_mapped'] = df['read_name'].isin(perf['read_name'].unique())
df['is_mapped'].value_counts(normalize=True)

In [ ]:
pdf = df.copy()
print(f"{pdf.shape=}")

mask = (pdf['n_enzymes'] > 0)
pdf = pdf[mask]

mask = (pdf['n_barcode_forward'].isin([0, 1]))
pdf = pdf[mask]

mask = (pdf['n_barcode_reverse_comp'].isin([0, 1]))
pdf = pdf[mask]

mask = (pdf['n_barcode_forward'] != 0) | (pdf['n_barcode_reverse_comp'] != 0)
pdf = pdf[mask]


print(f"{pdf.shape=}")


pdf.head()

In [ ]:
perf = res[res['mapping_quality'] > 0]

pdf['is_mapped'] = pdf['read_name'].isin(perf['read_name'].unique())
pdf['is_mapped'].value_counts(normalize=True)

In [ ]:
x

In [ ]:
# pdf = df.copy()
# sns.boxplot(data=pdf, 
#             x='n_enzymes',
#             y='mean_base_quality')